### Agentic RAG

In [1]:
import os

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from dotenv import load_dotenv

load_dotenv(override=True)

True

### Import Business Client & SQL Query Tools

In [2]:
import sys
from pathlib import Path

# Add parent directory to path to import tools
sys.path.insert(0, str(Path.cwd().parent))

# Import business client retrieval tools (CRM database)
from customers_management.customer_tools import business_client_tools

# Import SQL query tool for Chinook music database
from customers_management.sql_query_tool import chinook_sql_tool

print(f"Loaded {len(business_client_tools)} business client tools (CRM):")
for t in business_client_tools:
    print(f"  - {t.name}")

print(f"\nLoaded SQL query tool (Music Store):")
print(f"  - {chinook_sql_tool.name}")

Loaded 6 business client tools (CRM):
  - get_business_client_by_id
  - get_business_client_by_email
  - get_all_business_clients
  - get_active_business_clients
  - search_business_clients
  - get_business_client_count

Loaded SQL query tool (Music Store):
  - query_music_database


In [3]:
@tool("DuckDuckGoSearch")
def duckduckgo_search(query_string: str):
    """
    Useful to search for any kinds of information and
    when you need to search the internet for any kinds of information, use this tool.
    Prefer this tool when you search for long queries.
    Should not be used for Article search or Topic Search.
    """
    
    search = DuckDuckGoSearchAPIWrapper()
    
    return search.run(query_string)

In [4]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
wiki = WikipediaQueryRun(
    name="WikiepdiaSearch",
    description="Use this tool when you want to analyze for information on Wikipedia by Terms, Keywords or any Topics.",
    api_wrapper=api_wrapper)

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
).split_documents(docs)

vectordatabase = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordatabase.as_retriever()

In [7]:
from langchain_classic.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "search for information about langsmith. for any questions related to langsmith, you must use this tool"
)

retriever_tool.name

'langsmith_search'

In [8]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [9]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [10]:
# Combine all tools: existing tools + business client tools + SQL query tool
tools = [arxiv, duckduckgo_search, wiki, retriever_tool] + business_client_tools + [chinook_sql_tool]

print(f"Total tools available: {len(tools)}")
print("\nAll tools:")
for i, t in enumerate(tools, 1):
    print(f"{i}. {t.name}")

tools

Total tools available: 11

All tools:
1. arxiv
2. DuckDuckGoSearch
3. WikiepdiaSearch
4. langsmith_search
5. get_business_client_by_id
6. get_business_client_by_email
7. get_all_business_clients
8. get_active_business_clients
9. search_business_clients
10. get_business_client_count
11. query_music_database


[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=1000)),
 StructuredTool(name='DuckDuckGoSearch', description='Useful to search for any kinds of information and\nwhen you need to search the internet for any kinds of information, use this tool.\nPrefer this tool when you search for long queries.\nShould not be used for Article search or Topic Search.', args_schema=<class 'langchain_core.utils.pydantic.DuckDuckGoSearch'>, func=<function duckduckgo_search at 0x00000121E10A5940>),
 WikipediaQueryRun(name='WikiepdiaSearch', description='Use this tool when you want to analyze for information on Wikipedia by Terms, Keywords or any Topics.', api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' fro

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    max_tokens=2000,
    temperature=0.1,
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

In [12]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("hwchase17/openai-functions-agent")

prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [13]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage

agent_executor = create_react_agent(
    llm,
    tools = tools
)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14028\3333576309.py:4: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(


In [14]:
# for stream in agent_executor.stream({
#     "messages": [
#         HumanMessage(content="tell me about langsmith")
#     ]
# }):
#     print(stream)
#     print("***********")

In [15]:
# for stream in agent_executor.stream({
#     "messages": [
#         HumanMessage(content="whats the paper 2412.16446 talk about it?")
#     ]
# }):
#     print(stream)
#     print("***********")

In [16]:
# for stream in agent_executor.stream({
#     "messages": [
#         HumanMessage(content="Indian Constitution")
#     ]
# }):
#     print(stream)
#     print("***********")

In [17]:
# for stream in agent_executor.stream({
#     "messages": [
#         HumanMessage(content="Who won the world cup in the Year 2025 in ICC?")
#     ]
# }):
#     print(stream)
#     print("***********")

### Business Client Tools Demo (CRM Database)

Test the business client retrieval tools integrated with the agent

In [32]:
# Test 1: Get business client count
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="How many business clients are in the CRM database?")
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 1200, 'total_tokens': 1212, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9e416ec019', 'id': 'chatcmpl-D87e2R28icNElSo8jdmyvxpz34ONu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c4d98-c062-7490-bbf7-ba74bbf121f5-0', tool_calls=[{'name': 'get_business_client_count', 'args': {}, 'id': 'call_YnSNNcsod4b8PTUc1L2r93gE', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 1200, 'output_tokens': 12, 'total_tokens': 1212, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reason

In [19]:
# # Test 2: Search for business clients
# for stream in agent_executor.stream({
#     "messages": [
#         HumanMessage(content="Find all business clients with the name 'Patel'")
#     ]
# }):
#     print(stream)
#     print("***********")

In [20]:
# # Test 3: Get business client by ID
# for stream in agent_executor.stream({
#     "messages": [
#         HumanMessage(content="Show me the details of business client with ID 1")
#     ]
# }):
#     print(stream)
#     print("***********")

In [21]:
# # Test 4: Get business client by email
# for stream in agent_executor.stream({
#     "messages": [
#         HumanMessage(content="Find the business client with email pari.patel365@yahoo.in")
#     ]
# }):
#     print(stream)
#     print("***********")

In [22]:
# # Test 5: Get all active business clients
# for stream in agent_executor.stream({
#     "messages": [
#         HumanMessage(content="Show me all active business clients")
#     ]
# }):
#     print(stream)
#     print("***********")

In [23]:
# # Test 6: Complex query combining business clients with other tools
# for stream in agent_executor.stream({
#     "messages": [
#         HumanMessage(content="How many active business clients do we have in our CRM? Also tell me about Indian Constitution using Wikipedia.")
#     ]
# }):
#     print(stream)
#     print("***********")

### Chinook Music Database SQL Query Tool Demo

Test the SQL query tool that handles complex analytical questions about the music store

In [24]:
# Test 1: Simple count query
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="How many artists are in the music database?")
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 1199, 'total_tokens': 1222, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9e416ec019', 'id': 'chatcmpl-D87cZ1SkpEfrmOFfzYcdjZZkVUpd0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c4d97-5b9e-75d0-8dc3-e3edc4443f78-0', tool_calls=[{'name': 'query_music_database', 'args': {'question': 'How many artists are in the music database?'}, 'id': 'call_slZMaBlu3dx0n53BHdmDCX0X', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 1199, 'output_tokens': 23, 'total_tokens': 1222, 'input_token_details': {'audio': 0, 'cache_rea

In [29]:
# Test 2: Top artists query
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Which artist has the most albums in the database?")
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 1200, 'total_tokens': 1221, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9e416ec019', 'id': 'chatcmpl-D87cvZcwwjmrkjPSAtgAbw8CP9iTh', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c4d97-b35c-7180-87ea-8e0756e4ff64-0', tool_calls=[{'name': 'query_music_database', 'args': {'question': 'Which artist has the most albums?'}, 'id': 'call_tCkmB8eyOBG34YOagLqZzzBW', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 1200, 'output_tokens': 21, 'total_tokens': 1221, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'o

In [30]:
# Test 3: Customer spending analysis
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Which country's customers have spent the most? Give me the top 5 with amounts.")
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 1207, 'total_tokens': 1238, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9e416ec019', 'id': 'chatcmpl-D87dMKBWMZmbpR5tzwKU0p5pBT89k', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c4d98-1ae1-7293-8738-6ec577471b96-0', tool_calls=[{'name': 'query_music_database', 'args': {'question': "Which country's customers have spent the most? Give me the top 5 with amounts."}, 'id': 'call_Ll5rSzttq7lSUeIdHUP6ZQQv', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 1207, 'output_tokens': 31, 'total_tokens': 1238, 'input_toke

In [27]:
# # Test 4: Genre analysis
# for stream in agent_executor.stream({
#     "messages": [
#         HumanMessage(content="What are the most popular music genres based on number of tracks?")
#     ]
# }):
#     print(stream)
#     print("***********")

In [31]:
# Test 5: Complex multi-tool query
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Tell me about the top selling artist in the music database. Also, search Wikipedia to tell me about their history.")
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 1213, 'total_tokens': 1276, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9e416ec019', 'id': 'chatcmpl-D87dflFdSi5W2L1O3fHlSh1ZK71PJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c4d98-63c0-7740-8871-13f535abf2c0-0', tool_calls=[{'name': 'query_music_database', 'args': {'question': 'Who is the top selling artist in the music database?'}, 'id': 'call_FXbIgAXIrkMf96tDMmOPATYz', 'type': 'tool_call'}, {'name': 'WikiepdiaSearch', 'args': {'query': 'top selling music artist history'}, 'id': 'call_5ZbakdVYiVq7ZJoPsDVgIvdu', 'type': 'tool_call'}